## Running a bell circuit on different backends

In [1]:
import numpy as np
import time

from braket.circuits import Circuit
from braket.devices import LocalSimulator
from qiskit_braket_provider import AWSBraketProvider
from qiskit_braket_provider.providers import adapter
from braket.aws import AwsSession

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.primitives import BackendEstimator, Estimator
from qiskit.quantum_info.operators import SparsePauliOp
from qiskit.primitives import BackendEstimator
from qiskit.circuit import ParameterVector, Parameter

import sys
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
module_path = os.path.abspath('~/Downloads/min_example_hybridjob/BraketEstimator')
if module_path not in sys.path:
    sys.path.append(module_path)

from BraketEstimator.braket_estimator import BraketEstimator

aws_session = AwsSession(default_bucket="amazon-braket-us-west-1-lukasvoss")
AWSBraketProvider().backends()

[BraketBackend[Aria 1],
 BraketBackend[Aria 2],
 BraketBackend[Aspen-M-3],
 BraketBackend[Forte 1],
 BraketBackend[Harmony],
 BraketBackend[Lucy],
 BraketBackend[SV1],
 BraketBackend[TN1],
 BraketBackend[dm1]]

In [2]:
batchsize = 50
n_shots = 10_000

#### Run with native Braket SV Backend

In [3]:
backend = AWSBraketProvider().get_backend('SV1')

device = backend._device
device

Device('name': SV1, 'arn': arn:aws:braket:::device/quantum-simulator/amazon/sv1)

In [4]:
braket_bell_circ = Circuit().h(0).cnot(0, 1)
print(braket_bell_circ)

T  : |0|1|
          
q0 : -H-C-
        | 
q1 : ---X-

T  : |0|1|


In [5]:
start_time = time.time()
job_braket_native = device.run_batch(
    [braket_bell_circ] * batchsize,
    shots=n_shots,
)
job_braket_native.results()
run_batch_time = time.time() - start_time
print('--- {} seconds ---'.format(round(run_batch_time, 2)))
print('Execution time per circuit:', round(run_batch_time / batchsize, 2), 'seconds')

--- 20.18 seconds ---
Execution time per circuit: 0.4 seconds


#### Run with Braket Provider + **Estimator Primitive**

In [6]:
qiskit_bell_circ = QuantumCircuit(2)
qiskit_bell_circ.h(0)
qiskit_bell_circ.cx(0, 1)

qiskit_bell_circ.draw()

┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

In [16]:
backend = AWSBraketProvider().get_backend('SV1')
estimator = BackendEstimator(backend)
qiskit_observables = SparsePauliOp.from_list(([('XX', 0.25), ("IZ", 0.25), ("ZZ", 0.25), ("XY", 0.25)]))

qiskit_start_time = time.time()
job = estimator.run(
    circuits=[qiskit_bell_circ] * batchsize,
    observables=[qiskit_observables] * batchsize,
    shots=n_shots
)
job.result().values # This is the time-consuming line

qiskit_run_batch_time = time.time() - qiskit_start_time
print('--- {} seconds ---'.format(round(qiskit_run_batch_time, 2)))
print('Execution time per circuit:', round(qiskit_run_batch_time / (batchsize * 3), 2), 'seconds')

Start: 2024-02-20 15:56:05.135379
Execution: 2024-02-20 15:56:48.138247 <class 'qiskit_braket_provider.providers.braket_job.AmazonBraketTask'>
Download: 2024-02-20 15:57:08.745333
--- 64.19 seconds ---
Execution time per circuit: 0.43 seconds


In [8]:
qiskit_expvals = job.result().values

#### Use ``Estimator`` from Qiskit

In [9]:
start_time = time.time()
job = Estimator().run(
    circuits=[qiskit_bell_circ] * batchsize,
    observables=[qiskit_observables] * batchsize,
    shots=n_shots
)
qiskit_expvals_2 = job.result().values
run_batch_time = time.time() - start_time
print('--- {} seconds ---'.format(round(run_batch_time, 2)))
# Three non-commuting parts of the Hamiltonian lead to three measured circuits per Hamiltonian
print('Execution time per circuit:', round(run_batch_time / (batchsize * 3), 2), 'seconds')

--- 0.02 seconds ---
Execution time per circuit: 0.0 seconds


In [10]:
qiskit_expvals_2 - qiskit_expvals

array([-0.00906679, -0.00443542, -0.0065087 , -0.0011198 ,  0.0045902 ,
        0.00071705, -0.00243419,  0.005337  , -0.00478905, -0.00311659,
        0.00291255,  0.00623553, -0.00010261, -0.00320417,  0.00275132,
       -0.00102037, -0.00483031,  0.00994051,  0.00463327, -0.00769813,
       -0.00048143,  0.00136966,  0.00333032, -0.0092636 ,  0.00688094,
       -0.00260573,  0.00502991, -0.00916265, -0.00443699,  0.00981192,
        0.00244041, -0.00197711, -0.00230018, -0.00513607, -0.00464422,
        0.0011848 , -0.00287507,  0.00256599,  0.00573341,  0.01119874,
       -0.00665702, -0.0030332 , -0.00581848,  0.0035563 ,  0.00308019,
       -0.00279815,  0.00402483,  0.00615955, -0.00367583, -0.00067999])

In [11]:
max(qiskit_expvals_2 - qiskit_expvals)

0.011198743356155116

#### Run Circuit and Measure observables on own ``BraketEstimator``

In [12]:
backend = LocalSimulator()
estimator = BraketEstimator(backend)

braket_observables = qiskit_observables.to_list()

In [13]:
start_time = time.time()
expvals = estimator.run(
    circuit=[braket_bell_circ] * batchsize,
    observables=[braket_observables] * batchsize,
    target_register=[[0, 1]] * batchsize,
    shots=n_shots
)
run_braket_estimator_time = time.time() - start_time
print('--- {} seconds ---'.format(round(run_braket_estimator_time, 2)))
print('Execution time per circuit:', round(run_braket_estimator_time / (batchsize * 3), 2), 'seconds')

TypeError: to_unsigned_observable() got an unexpected keyword argument 'include_trivial'

In [ ]:
expvals.astype(float) - qiskit_expvals

/var/folders/6b/76tj41f506g7_0dg_k15w_d80000gn/T/ipykernel_22368/798129361.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  expvals.astype(float) - qiskit_expvals


array([ 9.450e-03,  1.200e-03,  9.800e-03,  4.800e-03, -8.500e-04,
        2.450e-03,  3.250e-03,  3.300e-03, -7.000e-03, -5.000e-05,
        7.250e-03,  8.000e-04,  3.400e-03,  2.050e-03, -8.150e-03,
        6.000e-04,  3.750e-03,  3.400e-03, -2.300e-03,  1.600e-03,
       -4.800e-03,  6.500e-03,  1.150e-03,  1.100e-03, -5.850e-03,
       -5.500e-04,  3.000e-04, -5.000e-04,  4.800e-03,  2.200e-03,
       -5.600e-03, -3.250e-03, -3.500e-04, -6.000e-04, -6.000e-04,
        4.800e-03, -6.200e-03, -2.850e-03,  2.350e-03,  1.135e-02,
        3.550e-03,  1.300e-03, -3.400e-03,  6.450e-03,  1.340e-02,
        3.050e-03, -1.050e-03,  2.950e-03, -9.300e-03,  8.300e-03])

In [ ]:
max(expvals.astype(float) - qiskit_expvals)

/var/folders/6b/76tj41f506g7_0dg_k15w_d80000gn/T/ipykernel_22368/3208180139.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  max(expvals.astype(float) - qiskit_expvals)


0.013400000000000079

#### Running batch of qiskit circuits with SV1 backend provider

In [14]:
backend = AWSBraketProvider().get_backend('SV1')

In [15]:
qiskit_start_time = time.time()
job = backend.run(
    [qiskit_bell_circ] * batchsize,
    shots=n_shots
)
result = job.result() # This is the time-consuming line

qiskit_run_batch_time = time.time() - qiskit_start_time
print('--- {} seconds ---'.format(round(qiskit_run_batch_time, 2)))
print('Execution time per batch:', round(qiskit_run_batch_time / batchsize, 2), 'seconds')

--- 21.37 seconds ---
Execution time per batch: 0.43 seconds
